In [1]:
import os
import pandas as pd
import pywencai
from tqdm import tqdm
import sys
import tushare as ts
sys.path.append(r'D:\作业\实习工作相关')
from junwei_util import *     #这是我自己常用的工具函数
from tqdm import tqdm
from datetime import datetime
date_list=pd.read_excel("日历（上海）.xlsx")
date_list= date_list['day'].dt.strftime('%Y%m%d').tolist()
date_list=[i for i in date_list if i >= '20241008']
print(date_list)
query='20240930下跌 20241008上涨 20241008热度排名前10 20241009上涨 20241010涨幅'#f'{date_list[0]}下跌 {date_list[1]}上涨 {date_list[1]}热度排名小于150 {date_list[2]}上涨 {date_list[3]}涨幅'
print(query)
df=pywencai.get(query=query, perpage=50, sort_order='asc', query_type='stock', loop=True)
df.to_excel('实验覆盖.xlsx',index=False)
matching_columns = [col for col in df.columns if '涨跌幅' in col]
matching_columns.sort()
print(matching_columns)
df

['20241008', '20241009', '20241010', '20241014', '20241015', '20241016', '20241017', '20241018', '20241021', '20241022', '20241023', '20241024', '20241025', '20241028', '20241029', '20241030', '20241031', '20241101', '20241104', '20241105', '20241106', '20241107', '20241108', '20241111', '20241112', '20241113', '20241114', '20241115', '20241118', '20241119', '20241120', '20241121', '20241122', '20241125', '20241126', '20241127', '20241128', '20241129', '20241202', '20241203', '20241204', '20241205', '20241206', '20241209', '20241210', '20241211', '20241212', '20241213', '20241216', '20241217', '20241218', '20241219', '20241220', '20241223', '20241224', '20241227', '20241230', '20241231']
20240930下跌 20241008上涨 20241008热度排名前10 20241009上涨 20241010涨幅
['最新涨跌幅', '涨跌幅:前复权[20240930]', '涨跌幅:前复权[20241008]', '涨跌幅:前复权[20241009]', '涨跌幅:前复权[20241010]']


,股票代码,股票简称,最新价,最新涨跌幅,涨跌幅:前复权[20240930],涨跌幅:前复权[20241008],涨跌幅:前复权[20241009],涨跌幅:前复权[20241010],个股热度排名[20241008],个股热度排名排名[20241008],...,最低价:前复权[20240930],最低价:前复权[20241008],最低价:前复权[20241009],最低价:前复权[20241010],收盘价:前复权[20240930],收盘价:前复权[20241008],收盘价:前复权[20241009],收盘价:前复权[20241010],market_code,code
0,603007.SH,*ST花王,17.36,3.952095808,-2.48175182,3.29341317,5.07246377,4.96551724,2653,2653/5358,...,6.51,6.44,6.69,7.2,6.68,6.9,7.25,7.61,22,603007
1,000525.SZ,*ST红阳,NaN,NaN,-2.3557126,4.94571773,5.05747126,4.70459519,2690,2690/5358,...,8.07,8.3,8.82,9.2,8.29,8.7,9.14,9.57,33,000525


In [10]:
import re
def get_popular_stock(st, calendar,hot=50):
    # 1. 对 calendar 列表进行排序并筛选出比 st 大的元素
    temp=calendar[calendar.index(st)-1]
    calendar = sorted([date for date in calendar if date >=temp ])
    #print(len(calendar))
    # 2. 创建一个空的 DataFrame，设置列名
    merged_df = pd.DataFrame()

    # 3. 开始循环遍历 calendar 中的每个日期
    for i in tqdm(range(3,len(calendar))):
        # 构建 query 字符串
        
        query=f'{calendar[i-3]}下跌 {calendar[i-2]}上涨 {calendar[i-2]}热度排名前{hot} {calendar[i-1]}上涨 {calendar[i]}涨幅'
        #print(query)
        if calendar[i]>datetime.now().strftime('%Y%m%d'):
            continue
        # 4. 调用 pywencai.get() 获取数据
        df = pywencai.get(query=query, perpage=50, sort_order='asc', query_type='stock', loop=True)
        if  df is None:
            print(f"No data for {i}")
            continue
        #print(df)
        # 只保留指定的列
        required_columns = ['股票代码', '股票简称', '个股热度排名', '个股热度']
        
        duplicate_col=['涨跌幅','收盘价','开盘价','最低价','最高价']
        for col in duplicate_col:
            matching_columns = [column for column in df.columns if col in column and '最新' not in column]
            matching_columns.sort()
            
            # 调整列顺序
            df = df[[col for col in df.columns if col not in matching_columns] + matching_columns]
            
            # 给 matching_columns 列表中的列添加后缀 _n, _n+1, _n+2 ...
            new_column_names = [f"{matching_columns[0]}_{i}" for i in range(len(matching_columns))]
            rename_mapping = dict(zip(matching_columns, new_column_names))
            df = df.rename(columns=rename_mapping)

            required_columns.extend(new_column_names)
        df = df[[col for col in df.columns if any(keyword in col for keyword in required_columns)]]


        # 遍历列名并清理，如果列名包含 '[' 则去除后续部分
        # df.columns = [col.split('[')[0] if '[' in col else col for col in df.columns]
        df.columns = [re.sub(r'\[.*?\]', '', col) for col in df.columns]

        # 创建 "交易日期" 列并赋值为 require_time
        df['n+4天涨跌幅日期'] = calendar[i]
        df['n+2天热度排名日期']= calendar[i-2]
        df['查询条件'] = query

        # 5. 合并 df 和 merged_df
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    merged_df.sort_values(by=['n+4天涨跌幅日期','个股热度排名'], inplace=True)

    # 6. 返回合并后的 DataFrame
    return merged_df

date_list=pd.read_excel("日历（上海）.xlsx")
date_list= date_list['day'].dt.strftime('%Y%m%d').tolist()
a=get_popular_stock("20241008",date_list)
a

100%|██████████| 56/56 [01:03<00:00,  1.13s/it]


,股票代码,股票简称,个股热度排名,个股热度排名排名,个股热度,涨跌幅:前复权_0,涨跌幅:前复权_1,涨跌幅:前复权_2,涨跌幅:前复权_3,收盘价:前复权_0,...,最低价:前复权_1,最低价:前复权_2,最低价:前复权_3,最高价:前复权_0,最高价:前复权_1,最高价:前复权_2,最高价:前复权_3,n+4天涨跌幅日期,n+2天热度排名日期,查询条件
0,603007.SH,*ST花王,2653,2653/5358,252677.5,-2.48175182,3.29341317,5.07246377,4.96551724,6.68,...,6.44,6.69,7.2,6.81,6.97,7.25,7.61,20241010,20241008,20240930下跌 20241008上涨 20241008热度排名前50 20241009...
1,000525.SZ,*ST红阳,2690,2690/5358,247267.0,-2.3557126,4.94571773,5.05747126,4.70459519,8.29,...,8.3,8.82,9.2,8.55,8.7,9.14,9.6,20241010,20241008,20240930下跌 20241008上涨 20241008热度排名前50 20241009...
2,000908.SZ,*ST景峰,340,340/5359,1872147.0,-4.98533724,4.9382716,5,5.06666667,3.24,...,3.25,3.57,3.94,3.42,3.4,3.57,3.94,20241014,20241009,20241008下跌 20241009上涨 20241009热度排名前50 20241010...
3,600298.SH,安琪酵母,630,630/5359,1080668.0,-0.49423394,2.37306843,1.45552561,-0.80688542,36.24,...,34.61,37.09,36.03,40,37.99,38.88,37.14,20241014,20241009,20241008下跌 20241009上涨 20241009热度排名前50 20241010...
4,002564.SZ,*ST天沃,2093,2093/5359,297824.0,-2.18068536,2.86624204,3.71517028,1.22324159,3.14,...,3.03,3.15,3.18,3.37,3.28,3.39,3.32,20241014,20241009,20241008下跌 20241009上涨 20241009热度排名前50 20241010...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,300131.SZ,英唐智控,832,832/5371,1042900.0,-2.52004582,1.41010576,0.34762457,None,8.51,...,8.31,8.56,8.66,9.05,8.78,8.81,8.66,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...
1170,300654.SZ,世纪天鸿,842,842/5371,1024476.5,-3.79278446,7.21153846,17.93721973,0.38022814,10.40,...,10.29,11.1,12.29,10.84,11.16,13.38,14.4,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...
1171,002779.SZ,中坚科技,853,853/5371,1008062.5,-1.53628462,9.99794703,1.92235909,-7.54440579,48.71,...,47.51,54.26,50.2,50.99,53.58,58.55,59,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...
1172,600958.SH,东方证券,860,860/5371,1001504.0,-2.97202797,0.9009009,0.71428571,-4.25531915,11.10,...,11.01,11.22,10.79,11.53,11.3,11.8,11.39,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...


In [11]:
a.to_excel('新策略.xlsx',index=False)

In [68]:
all_df=pd.read_excel(r"D:\作业\实习工作相关\清萍量化\新策略\新策略.xlsx")
all_df['n+4天涨跌幅日期']=all_df['n+4天涨跌幅日期'].astype(str)
all_df['n+2天热度排名日期']=all_df['n+2天热度排名日期'].astype(str)
print(len(all_df[all_df['涨跌幅:前复权_3']>-0.1])/len(all_df))
# 创建 all_data 的 label 列
all_df['label'] = all_df['涨跌幅:前复权_3'].apply(lambda x: 1 if x > 0 else 0)


all_df

0.516725352112676


,股票代码,股票简称,个股热度排名,个股热度排名排名,个股热度,涨跌幅:前复权_0,涨跌幅:前复权_1,涨跌幅:前复权_2,涨跌幅:前复权_3,开盘价:前复权_0,...,最低价:前复权_2,最低价:前复权_3,最高价:前复权_0,最高价:前复权_1,最高价:前复权_2,最高价:前复权_3,n+4天涨跌幅日期,n+2天热度排名日期,查询条件,label
0,600298.SH,安琪酵母,630,630/5359,1080668.0,-0.494234,2.373068,1.455526,-0.806885,39.90,...,37.09,36.03,40.00,37.99,38.88,37.14,20241014,20241009,20241008下跌 20241009上涨 20241009热度排名前50 20241010...,0
1,002600.SZ,领益智造,26,26/5360,10660061.0,-9.987670,9.041096,9.958506,-1.006289,7.78,...,6.97,7.76,7.79,8.03,7.95,8.19,20241015,20241010,20241009下跌 20241010上涨 20241010热度排名前50 20241014...,0
2,601318.SH,中国平安,31,31/5360,9779874.5,-9.768346,5.026930,1.963851,-2.522584,59.07,...,56.62,56.16,59.07,59.66,58.31,58.35,20241015,20241010,20241009下跌 20241010上涨 20241010热度排名前50 20241014...,0
3,000617.SZ,中油资本,40,40/5360,7890455.0,-9.987670,5.479452,10.035419,-8.047210,7.95,...,8.68,8.40,8.11,8.03,9.32,9.65,20241015,20241010,20241009下跌 20241010上涨 20241010热度排名前50 20241014...,0
4,000858.SZ,五粮液,47,47/5360,7253382.0,-9.978665,2.911701,0.073505,-3.846154,160.00,...,146.01,144.00,160.00,155.53,151.00,150.10,20241015,20241010,20241009下跌 20241010上涨 20241010热度排名前50 20241014...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,600734.SH,实达集团,1106,1106/5371,763457.0,-2.964960,1.666667,10.109290,-3.722084,3.66,...,3.62,3.82,3.74,3.81,4.03,4.29,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...,0
1132,601988.SH,中国银行,1153,1153/5371,732829.5,-0.415800,0.626305,0.414938,0.413223,4.81,...,4.78,4.79,4.85,4.83,4.86,4.90,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...,1
1133,002406.SZ,远东传动,1210,1210/5371,699605.5,-1.560284,0.864553,6.000000,-3.638814,7.08,...,6.91,7.09,7.09,7.03,7.64,7.31,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...,0
1134,000997.SZ,新大陆,1232,1232/5371,689711.5,-0.672430,2.852998,1.645510,-4.949121,21.01,...,20.95,20.43,21.65,21.32,21.96,22.62,20241115,20241113,20241112下跌 20241113上涨 20241113热度排名前50 20241114...,0


In [69]:
"""仅热度提高"""
popular=all_df[(all_df['个股热度排名']<=200) &(all_df['n+4天涨跌幅日期']>'20241101')]
#popular=popular.dropna()
#popular.to_csv('popular.csv',index=False)
print(len(popular[popular['涨跌幅:前复权_3']>-1])/len(popular))

0.5037593984962406


In [70]:
# 提取列名不包含 '日期' 且取值为数值型的列
numeric_columns = [
    col for col in popular.columns 
    if '日期' not in col and pd.api.types.is_numeric_dtype(popular[col])
]

# 计算相关性矩阵
correlation_matrix = popular[numeric_columns].corr()
correlation_matrix

,个股热度排名,个股热度,涨跌幅:前复权_0,涨跌幅:前复权_1,涨跌幅:前复权_2,涨跌幅:前复权_3,开盘价:前复权_0,开盘价:前复权_1,开盘价:前复权_2,开盘价:前复权_3,最低价:前复权_0,最低价:前复权_1,最低价:前复权_2,最低价:前复权_3,最高价:前复权_0,最高价:前复权_1,最高价:前复权_2,最高价:前复权_3,label
个股热度排名,1.000000,-0.710569,0.315978,-0.110982,-0.249445,0.059918,-0.094707,-0.096197,-0.104266,-0.111374,-0.094367,-0.093950,-0.100821,-0.105176,-0.103203,-0.102936,-0.045668,-0.111663,0.156199
个股热度,-0.710569,1.000000,-0.254863,0.061222,0.143794,-0.038674,0.030765,0.031862,0.037738,0.041415,0.030347,0.029129,0.034510,0.037436,0.038005,0.035827,-0.001798,0.041966,-0.091921
涨跌幅:前复权_0,0.315978,-0.254863,1.000000,0.043016,-0.011807,-0.008075,0.201471,0.215176,0.218805,0.216838,0.207466,0.217164,0.219974,0.220651,0.200299,0.212424,0.197953,0.222039,0.077835
涨跌幅:前复权_1,-0.110982,0.061222,0.043016,1.000000,0.271566,0.154429,-0.047313,-0.041653,-0.016555,-0.017449,-0.048176,-0.039768,-0.020395,-0.019866,-0.045658,-0.031210,-0.133011,-0.013734,0.126642
涨跌幅:前复权_2,-0.249445,0.143794,-0.011807,0.271566,1.000000,0.214071,-0.066087,-0.061623,-0.051054,-0.026779,-0.069116,-0.061660,-0.049829,-0.029828,-0.064037,-0.060070,-0.012236,-0.021924,0.193243
涨跌幅:前复权_3,0.059918,-0.038674,-0.008075,0.154429,0.214071,1.000000,-0.082986,-0.086255,-0.081471,-0.069657,-0.086460,-0.084385,-0.078902,-0.061764,-0.086328,-0.085902,-0.037557,-0.060793,0.819593
开盘价:前复权_0,-0.094707,0.030765,0.201471,-0.047313,-0.066087,-0.082986,1.000000,0.999231,0.998100,0.996831,0.999495,0.999177,0.998273,0.997223,0.999623,0.999124,0.995967,0.995773,-0.096045
开盘价:前复权_1,-0.096197,0.031862,0.215176,-0.041653,-0.061623,-0.086255,0.999231,1.000000,0.998982,0.997730,0.999576,0.999941,0.999115,0.997925,0.999280,0.999659,0.996717,0.996686,-0.095681
开盘价:前复权_2,-0.104266,0.037738,0.218805,-0.016555,-0.051054,-0.081471,0.998100,0.998982,1.000000,0.998783,0.998272,0.998915,0.999911,0.998893,0.998512,0.999429,0.998996,0.998234,-0.091197
开盘价:前复权_3,-0.111374,0.041415,0.216838,-0.017449,-0.026779,-0.069657,0.996831,0.997730,0.998783,1.000000,0.996532,0.997723,0.998966,0.999730,0.997368,0.998014,0.998203,0.999513,-0.082875


In [71]:
import tushare as ts
pro = ts.pro_api('20240522230128-a4912f53-70a9-4539-9ea6-dbc60b641bca')
pro._DataApi__http_url = 'http://tsapi.majors.ltd:7000'
all_df_groups=all_df.groupby('n+4天涨跌幅日期')
price_df=pd.DataFrame()
date_list=pd.read_excel("日历（上海）.xlsx")
date_list= date_list['day'].dt.strftime('%Y%m%d').tolist()
all_dates_n4 = all_df['n+4天涨跌幅日期'].astype(str).tolist()
all_dates_n2 = all_df['n+2天热度排名日期'].astype(str).tolist()

# 取三者的并集
date_list = list(set(date_list) | set(all_dates_n4) | set(all_dates_n2))

# 排列，使其按升序排列
date_list = sorted(date_list)
for date, group in tqdm(all_df_groups):
    # 获取date在date_list中的索引
    current_index = date_list.index(str(date))
    # 设置st (start_date)
    if current_index < 10:
        st = date_list[0]  # 如果是前10个日期之一，使用第一个日期
    else:
        st = date_list[current_index - 10]  # 否则使用当前日期前10个位置的日期  
    # 设置et (end_date)
    # if current_index >= len(date_list) - 3:
    #     et = date_list[-1]  # 如果是最后3个日期之一，使用最后一个日期
    # else:
    #     et = date_list[current_index + 3]  # 否则使用当前日期后3个位置的日期 
    et=date
    # 处理股票数据
    stock_list = group['股票代码'].tolist()
    stock_str = ','.join(stock_list)
    df = pro.daily(ts_code=stock_str, start_date=st, end_date=et)
    price_df = pd.concat([price_df, df], axis=0)
price_df.reset_index(inplace=True,drop=True)
price_df

100%|██████████| 25/25 [00:44<00:00,  1.79s/it]


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600298.SH,20241014,37.00,37.14,36.03,36.88,37.18,-0.30,-0.8069,128070.05,469025.008
1,600298.SH,20241011,37.67,38.10,36.75,37.18,37.64,-0.46,-1.2221,151100.87,564220.418
2,600298.SH,20241010,37.09,38.88,37.09,37.64,37.10,0.54,1.4555,287247.43,1087878.309
3,600298.SH,20241009,35.20,37.99,34.61,37.10,36.24,0.86,2.3731,413002.79,1514809.933
4,600298.SH,20241008,39.90,40.00,35.01,36.24,36.42,-0.18,-0.4942,485099.57,1805677.724
...,...,...,...,...,...,...,...,...,...,...,...
12881,603153.SH,20241101,18.60,19.19,18.24,18.98,18.71,0.27,1.4431,131066.69,247630.239
12882,603662.SH,20241101,33.33,34.50,33.10,33.28,33.51,-0.23,-0.6864,178592.25,601824.995
12883,603663.SH,20241101,17.50,18.88,17.30,18.00,17.61,0.39,2.2147,379844.18,690094.787
12884,603698.SH,20241101,15.75,15.89,14.98,15.01,15.85,-0.84,-5.2997,76982.39,117979.306


In [72]:
from tqdm import tqdm

# 示例初始化（替换为实际数据）
all_df_groups = all_df.groupby('n+4天涨跌幅日期')
price_df['vol_change_pct'] = np.nan
# 遍历日期和分组
for date, group in tqdm(all_df_groups):
    # 获取 date 在 date_list 中的索引
    current_index = date_list.index(str(date))
    
    # 设置 st (start_date)
    if current_index < 10:
        st = date_list[0]  # 如果是前 10 个日期之一，使用第一个日期
    else:
        st = date_list[current_index - 10]  # 否则使用当前日期前 10 个位置的日期  
    et=str(date)
    
    # # 设置 et (end_date)
    # if current_index >= len(date_list) - 3:
    #     et = date_list[-1]  # 如果是最后 3 个日期之一，使用最后一个日期
    # else:
    #     et = date_list[current_index + 3]  # 否则使用当前日期后 3 个位置的日期 
    
    # 处理股票数据
    stock_list = group['股票代码'].tolist()
    time_df = price_df[(price_df['trade_date'] >= st) & (price_df['trade_date'] <= et) & (price_df['ts_code'].isin(stock_list))]
    
    # 计算 vol 变化百分比
    for stock in stock_list:
        stock_df = time_df[time_df['ts_code'] == stock].copy()
        
        # 计算变化百分比
        stock_df['vol_change_pct'] = (stock_df['vol'] - stock_df['vol'].shift(-1)) / stock_df['vol'].shift(-1)
        
        # 更新到 price_df
        price_df.update(stock_df)

# 删掉由于无法计算变化百分比而产生缺失值的行
price_df = price_df.dropna(subset=['vol_change_pct'])

# 重置索引
price_df.reset_index(drop=True, inplace=True)
price_df.to_excel('price_df.xlsx', index=False)
price_df


100%|██████████| 25/25 [00:09<00:00,  2.60it/s]


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,vol_change_pct
0,600298.SH,20241014,37.00,37.14,36.03,36.88,37.18,-0.30,-0.8069,128070.05,469025.008,-0.152420
1,600298.SH,20241011,37.67,38.10,36.75,37.18,37.64,-0.46,-1.2221,151100.87,564220.418,-0.473970
2,600298.SH,20241010,37.09,38.88,37.09,37.64,37.10,0.54,1.4555,287247.43,1087878.309,-0.304490
3,600298.SH,20241009,35.20,37.99,34.61,37.10,36.24,0.86,2.3731,413002.79,1514809.933,-0.148623
4,600298.SH,20241008,39.90,40.00,35.01,36.24,36.42,-0.18,-0.4942,485099.57,1805677.724,0.251903
...,...,...,...,...,...,...,...,...,...,...,...,...
12096,603153.SH,20241104,19.35,20.88,18.98,20.57,18.98,1.59,8.3772,256509.81,515886.268,0.957094
12097,603662.SH,20241104,35.00,36.61,35.00,36.61,33.28,3.33,10.0060,91483.99,331085.924,-0.487749
12098,603663.SH,20241104,18.00,18.10,17.03,17.35,18.00,-0.65,-3.6111,312058.35,546798.107,-0.178457
12099,603698.SH,20241104,14.93,15.23,14.78,15.15,15.01,0.14,0.9327,43002.83,64826.817,-0.441394


In [ ]:



    # 合并 price_df 和 all_data
merged_df = price_df.merge(
    all_df,
    left_on=['ts_code', 'trade_date'],
    right_on=['股票代码', 'n+4天涨跌幅日期'],
    how='left'  # 左连接，保留 price_df 的完整行
)
merged_df.to_csv(f'ts and wencai.csv', index=False)

In [ ]:
dataset=pd.DataFrame()
all_list=[]
for date, group in tqdm(all_df_groups):
    # 获取 date 在 date_list 中的索引
    current_index = date_list.index(str(date))
    
    # 设置 st (start_date)
    if current_index < 6:
        st = date_list[0]  # 如果是前 10 个日期之一，使用第一个日期
    else:
        st = date_list[current_index - 6]  # 否则使用当前日期前 10 个位置的日期  
    et=str(date)
    time_df = merged_df[(merged_df['trade_date'] >= st) & (merged_df['trade_date'] <= et)]
    stock_list = list(set(time_df['ts_code']))
    for stock in stock_list:
        tem_list=[]
        stock_df = time_df[time_df['ts_code'] == stock]
        tem_list.extend(list(stock_df['vol_change_pct'])[1:])
        # 查找 trade_date 等于 et 的行并提取指定列
        row = stock_df[stock_df['trade_date'] == et]

        if not row.empty:  # 确保找到符合条件的行
            # 提取指定列的值
            extracted_values = row[['个股热度排名', '个股热度', '涨跌幅:前复权_0', '涨跌幅:前复权_1',
                                    '涨跌幅:前复权_2', 'label', '涨跌幅:前复权_3','ts_code','n+4天涨跌幅日期']].iloc[0].tolist()
            tem_list.extend(extracted_values)
            all_list.append(tem_list)

        
list_lengths = [len(lst) for lst in all_list]
 # 检查 all_list 中所有列表的长度是否相同
if all(len(lst) == len(all_list[0]) for lst in all_list):
    print("All lists in all_list have the same length.")
else:
    # 输出长度不相同的元素
    differing_elements = [lst for lst in all_list if len(lst) != list_lengths[0]]
    print("The lists in all_list do not have the same length. Differing elements:", differing_elements)


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
import pandas as pd
import matplotlib.pyplot as plt

# 示例 all_list 的数据验证一致性（假设 all_list 已知）
# 检查是否所有列表长度相同
all_list = [[1, 2, 3, 0], [4, 5, 6, 1], [7, 8, 9, 0]]  # 示例数据
if all(len(lst) == len(all_list[0]) for lst in all_list):
    # 转化为 DataFrame
    dataset = pd.DataFrame(all_list)

    # 指定 feature 和 label
    feature_cols = dataset.columns[:-3]  # 第一列到倒数第4列
    label_col = dataset.columns[-3]      # 倒数第三列

    X = dataset[feature_cols]
    y = dataset[label_col]

    # 创建决策树模型并训练
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X, y)

    # 输出模型拟合结果
    print("Model Training Score:", clf.score(X, y))
    print("Decision Tree Rules:\n")
    print(export_text(clf, feature_names=[f"Feature_{i}" for i in feature_cols]))

    # 绘制决策树
    plt.figure(figsize=(12, 8))
    plot_tree(clf, feature_names=[f"Feature_{i}" for i in feature_cols], class_names=["Class 0", "Class 1"], filled=True)
    plt.title("Decision Tree")
    plt.show()
else:
    print("The lists in all_list do not have the same length. Unable to proceed.")

